# Kiva Loan Data EDA and Analysis

### About Kiva
Kiva is an international nonprofit founded in 2005 with a mission to connect people through lending to alleviate poverty. In June 2018 Kiva was in 85 countries, and had served 2.9 Million borrowers through $ 1.16 Billion worth of loans.

### How it works
1. A borrower applies for a loan (micro-finance partner, or Kiva directly).
2. The loan goes through the underwriting and approval process.
3. The loan is posted to Kiva for lenders to support, and a 30-day fundraising period begins.
4. Borrowers repay the loans.
* Note that Kiva does not collect interest on loans, and Kiva lenders do not receive interest from loans they support on Kiva. However, Field Partners collect interest from borrowers to cover their operation costs.

### About Kiva's Data
This analysis uses Kiva's Data Snapshot (https://build.kiva.org/docs/data/snapshots) downloaded on June 15th, 2018.


## Libraries and Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sts
%matplotlib inline

In [2]:
!ls kiva_ds_csv

lenders.csv       loans.csv         loans_lenders.csv


In [3]:
lenders = pd.read_csv('kiva_ds_csv/lenders.csv')

In [4]:
loans = pd.read_csv('kiva_ds_csv/loans.csv', parse_dates=['POSTED_TIME', 'PLANNED_EXPIRATION_TIME', 'DISBURSE_TIME',
       'RAISED_TIME'])

In [5]:
loans_lenders = pd.read_csv('kiva_ds_csv/loans_lenders.csv')

In [5]:
lenders.isnull().sum()

PERMANENT_NAME             0
DISPLAY_NAME            2768
MAIN_PIC_ID          1590116
CITY                 1619306
STATE                1713481
COUNTRY_CODE         1458635
MEMBER_SINCE               0
PERSONAL_URL         2189805
OCCUPATION           1844514
LOAN_BECAUSE         2174852
OTHER_INFO           2154640
LOAN_PURCHASE_NUM     894281
INVITED_BY           1852349
NUM_INVITED                0
dtype: int64

In [6]:
lenders.shape

(2349174, 14)

In [7]:
loans.shape

(1419607, 34)

In [10]:
loans_lenders.shape

(1387432, 2)

In [9]:
test = loans_lenders.sample(10)

In [15]:
test

,LOAN_ID,LENDERS
333766,913663,"bryan1553, kbk, rolf5730, gertjan9162, thomas5..."
575889,901155,"jacqueline4222, trolltech4460, stephen30137852..."
166831,1121282,"karen5774, rich2817, martina8330, ted3130, jus..."
629627,849934,"dishforum1163, shannon40082015, nigel4691, kat..."
402234,446593,"paul23765609, ricardopereira, markdeac, mary34..."
1255565,629687,"ali9555, dylan5948, larryandkitty3677, cierra8..."
683168,42339,"ssnyder, chrispadfield, jennifer7131, collene,..."
1207236,1341837,"tim4327, fiona2159, richard3905, sander3009, d..."
1124816,618557,"josy6466, r3922, zm5282, laree2227, g7537, amy..."
1075189,72888,"cindy9335, john7424, doug7646, satoko8119, jea..."


In [12]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import numpy as np
from bs4 import BeautifulSoup
import string


#Step 1. Load the data
descriptions = test.LENDERS

#Step 2. Vectorize, Stem and Lemmatize words

vect = CountVectorizer()
model = vect.fit_transform(descriptions)

def tokenize(doc):
#     soup = BeautifulSoup(doc,'html.parser')
#     text = soup.get_text()
    translater = str.maketrans('','',string.punctuation)
    a = [word_tokenize(descriptions.translate(translater))]
    return a

# vect = TfidfVectorizer(analyzer='word',tokenizer=tokenize)
# model = vect.fit_transform(descriptions)
# features = vect.get_feature_names()

In [40]:
vect.get_feature_names()

['aaron83574977',
 'alexlee',
 'ali9555',
 'aliasgharmoiz',
 'amy3935',
 'amy7364',
 'andrea5814',
 'anita2681',
 'areef7365',
 'arun1891',
 'barbara6180',
 'betty95301705',
 'bob1643',
 'bryan1553',
 'caroline7000',
 'cathied9162',
 'chris4594',
 'chrisandcathy4269',
 'chrispadfield',
 'christchurchanglican5547',
 'christian6619',
 'christina3424',
 'cierra8791',
 'cindy9335',
 'claire1286',
 'clarence8190',
 'clorton',
 'collene',
 'dana5182',
 'daniel5404',
 'daniel9568',
 'danny6470',
 'danritz',
 'david3844',
 'david59642749',
 'david80737421',
 'david9667',
 'diane9275',
 'dishforum1163',
 'doug7646',
 'dylan5948',
 'edtracymaxgrace7258',
 'elias4241',
 'emily50205446',
 'emma5238',
 'enriquelopez',
 'eric2633',
 'esther9248',
 'evelyne9710',
 'fiona2159',
 'freddie1373',
 'g7537',
 'gary8860',
 'george23907283',
 'gertjan9162',
 'gooddogg1',
 'guy9829',
 'harold5454',
 'henry8741',
 'howard8651',
 'ijliao',
 'jacqueline4222',
 'james9460',
 'jandorfer',
 'jason7589',
 'jean5726'

In [42]:
vocs = vect.vocabulary_

In [48]:
vocs

{'aaron83574977': 0,
 'alexlee': 1,
 'ali9555': 2,
 'aliasgharmoiz': 3,
 'amy3935': 4,
 'amy7364': 5,
 'andrea5814': 6,
 'anita2681': 7,
 'areef7365': 8,
 'arun1891': 9,
 'barbara6180': 10,
 'betty95301705': 11,
 'bob1643': 12,
 'bryan1553': 13,
 'caroline7000': 14,
 'cathied9162': 15,
 'chris4594': 16,
 'chrisandcathy4269': 17,
 'chrispadfield': 18,
 'christchurchanglican5547': 19,
 'christian6619': 20,
 'christina3424': 21,
 'cierra8791': 22,
 'cindy9335': 23,
 'claire1286': 24,
 'clarence8190': 25,
 'clorton': 26,
 'collene': 27,
 'dana5182': 28,
 'daniel5404': 29,
 'daniel9568': 30,
 'danny6470': 31,
 'danritz': 32,
 'david3844': 33,
 'david59642749': 34,
 'david80737421': 35,
 'david9667': 36,
 'diane9275': 37,
 'dishforum1163': 38,
 'doug7646': 39,
 'dylan5948': 40,
 'edtracymaxgrace7258': 41,
 'elias4241': 42,
 'emily50205446': 43,
 'emma5238': 44,
 'enriquelopez': 45,
 'eric2633': 46,
 'esther9248': 47,
 'evelyne9710': 48,
 'fiona2159': 49,
 'freddie1373': 50,
 'g7537': 51,
 'g

In [16]:
test["LENDERS_lst"] = test["LENDERS"].map(lambda x: x.split(','))

In [18]:
test

,LOAN_ID,LENDERS,LENDERS_lst
333766,913663,"bryan1553, kbk, rolf5730, gertjan9162, thomas5...","[bryan1553, kbk, rolf5730, gertjan9162, th..."
575889,901155,"jacqueline4222, trolltech4460, stephen30137852...","[jacqueline4222, trolltech4460, stephen30137..."
166831,1121282,"karen5774, rich2817, martina8330, ted3130, jus...","[karen5774, rich2817, martina8330, ted3130,..."
629627,849934,"dishforum1163, shannon40082015, nigel4691, kat...","[dishforum1163, shannon40082015, nigel4691, ..."
402234,446593,"paul23765609, ricardopereira, markdeac, mary34...","[paul23765609, ricardopereira, markdeac, ma..."
1255565,629687,"ali9555, dylan5948, larryandkitty3677, cierra8...","[ali9555, dylan5948, larryandkitty3677, cie..."
683168,42339,"ssnyder, chrispadfield, jennifer7131, collene,...","[ssnyder, chrispadfield, jennifer7131, coll..."
1207236,1341837,"tim4327, fiona2159, richard3905, sander3009, d...","[tim4327, fiona2159, richard3905, sander300..."
1124816,618557,"josy6466, r3922, zm5282, laree2227, g7537, amy...","[josy6466, r3922, zm5282, laree2227, g7537..."
1075189,72888,"cindy9335, john7424, doug7646, satoko8119, jea...","[cindy9335, john7424, doug7646, satoko8119,..."


In [21]:
test = test.drop('LENDERS',axis=1)

In [23]:
lst_col = 'LENDERS_lst'
test_1 = pd.DataFrame({col:np.repeat(test[col].values, test[lst_col].str.len())for col in test.columns.difference([lst_col])}).assign(**{lst_col:np.concatenate(test[lst_col].values)})[test.columns.tolist()]

In [29]:
test_1 = test_1.drop('value',axis=1)

In [56]:
test_1.head()

,LOAN_ID,LENDERS_lst
0,913663,bryan1553
1,913663,kbk
2,913663,rolf5730
3,913663,gertjan9162
4,913663,thomas5363


In [54]:
test_1['LENDERS_lst'].map(lambda x: vocs[x])

KeyError: ' kbk'

In [58]:
df = pd.get_dummies(test_1.LENDERS_lst)

In [59]:
df['LOAN_ID'] = test_1.LOAN_ID

In [69]:
df2 = df.groupby('LOAN_ID').sum()

In [71]:
df2.head()

,JY1024,aaron83574977,alexlee,aliasgharmoiz,amy3935,amy7364,andrea5814,anita2681,areef7365,arun1891,...,ali9555,bryan1553,cindy9335,dishforum1163,jacqueline4222,josy6466,karen5774,paul23765609,ssnyder,tim4327
LOAN_ID,,,,,,,,,,,,,,,,,,,,,
42339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
72888,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
446593,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
618557,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
629687,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [84]:
sum(df2.iloc[1,:])

7

In [85]:
list(test[test['LOAN_ID'] == 72888].LENDERS_lst)

[['cindy9335',
  ' john7424',
  ' doug7646',
  ' satoko8119',
  ' jeanene5460',
  ' jonathan3172',
  ' daniel5404']]

In [30]:
from scipy.sparse import csr_matrix

In [ ]:
LOAN_ID = np.array([, 1, 1, 1, 1, 1])

In [35]:
data = np.array([1, 1, 1, 1, 1, 1])

In [36]:
csr_matrix((data, (test_1.LOAN_ID[:6],test_1.LENDERS_lst[:6])), shape=(3, 3)).toarray()

ValueError: invalid literal for int() with base 10: 'bryan1553'

In [32]:
csr_matrix(test_1.T.values)

TypeError: no supported conversion for types: (dtype('O'),)

In [27]:
test_1.pivot(index='LOAN_ID', columns='LENDERS_lst', values='value')

ValueError: Index contains duplicate entries, cannot reshape

In [4]:
samples = loans_lenders.sample(n=100)

In [6]:
samples["LENDERS"] = samples["LENDERS"].map(lambda x: x.split(','))

In [7]:
x=list(samples.LENDERS)

In [ ]:
y=np.array([np.array(xi) for xi in x])

In [ ]:
y.flatten().shape

In [ ]:
y[1]

In [ ]:
for idx,row in enumerate(samples["LENDERS"]):
    for user in row:
        if user not in set(list(samples.columns)):
            samples[user] = 0
            samples[user][idx]=1

In [ ]:
samples['new']=0

In [ ]:
samples['new'][1]=1

In [ ]:
samples.new.value_counts()

In [ ]:
df[['DESCRIPTION', 'DESCRIPTION_TRANSLATED','IMAGE_ID',
              'VIDEO_ID', 'NUM_JOURNAL_ENTRIES', 'TAGS', 'NUM_BULK_ENTRIES']].head()

In [ ]:
total = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total Missings', 'Percent Missings'])
missing_data

In [ ]:
# Drop columns we don't need
df = df.drop(['DESCRIPTION', 'DESCRIPTION_TRANSLATED','IMAGE_ID',
              'VIDEO_ID', 'NUM_JOURNAL_ENTRIES', 'TAGS', 'NUM_BULK_ENTRIES',
              ], axis=1)

In [ ]:
df.columns

In [ ]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
df.head()

In [ ]:
df.STATUS.value_counts()

In [ ]:
df['diff_posted_planned'] = df['PLANNED_EXPIRATION_TIME'].sub(df['POSTED_TIME'], axis=0)
df['diff_posted_planned'] = df['diff_posted_planned'] / np.timedelta64(1, 'D')

In [ ]:
df['diff_posted_raised'] = df['RAISED_TIME'].sub(df['POSTED_TIME'], axis=0)
df['diff_posted_raised'] = df['diff_posted_raised'] / np.timedelta64(1, 'D')

In [ ]:
df.head()

In [ ]:
# Empirical expiration time is about 103 days for direct loans
df[df['DISTRIBUTION_MODEL']=='direct']['diff_posted_planned'].mean()

In [ ]:
# Empirical expiration time is about 37 days for field-partner loans
df[df['DISTRIBUTION_MODEL']=='field_partner']['diff_posted_planned'].mean()

In [ ]:
# Average time (in days) it takes to fund direct loan
df[df['DISTRIBUTION_MODEL']=='direct']['diff_posted_raised'].mean()

In [ ]:
# Average time (in days) it takes to fund field-partner loan
df[df['DISTRIBUTION_MODEL']=='field_partner']['diff_posted_raised'].mean()

In [ ]:
# Checking for missing data
total = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total Missings', 'Percent Missings'])
missing_data

In [ ]:
df['REPAYMENT_INTERVAL'].value_counts(1)

In [ ]:
df['STATUS'].value_counts(1)

In [ ]:
df['DISTRIBUTION_MODEL'].value_counts(1)

In [ ]:
# Group vs Individual loans
# mask_g = df['BORROWER_GENDERS']!= 'male' & df['BORROWER_GENDERS'] != 'female'
df['GROUP'] = np.where(np.logical_and(df['BORROWER_GENDERS']!= 'male', df['BORROWER_GENDERS']!= 'female'), 1, 0)

In [ ]:
df.head()

## A closer look... Individual vs. Group loans

In [ ]:
individual = df['GROUP']==0
group = df['GROUP']==1

In [ ]:
df['GROUP'].value_counts(1)

In [ ]:
df[['FUNDED_AMOUNT', 'LOAN_AMOUNT', 'LENDER_TERM', 'NUM_LENDERS_TOTAL']][group].describe().T

In [ ]:
df[['FUNDED_AMOUNT', 'LOAN_AMOUNT', 'LENDER_TERM', 'NUM_LENDERS_TOTAL']][individual].describe().T

In [ ]:
# Distribution by group
plt.figure(figsize=(20,8))
sns.distplot(df[individual]['LOAN_AMOUNT'], hist=False, color='blue', label = 'Individual', kde_kws={'clip': (0, 8000)})
sns.distplot(df[group]['LOAN_AMOUNT'], hist=False, color='purple', label = 'Group', kde_kws={'clip': (0, 8000)});
plt.axvline(df[individual]['LOAN_AMOUNT'].mean(), color="blue", linestyle="--")
plt.axvline(df[group]['LOAN_AMOUNT'].mean(), color="purple", linestyle="--")

plt.legend();

In [ ]:
# H0: mu_group - mu_individual = 0
# Ha: mu_group - mu_induvidual != 0

# alpha = 0.05

# T test
t, p = sts.ttest_ind(df[individual]['LOAN_AMOUNT'], df[group]['LOAN_AMOUNT'], equal_var=False)

# p value
print("p-value: {:2.2f}".format(p))

## Individual loans, by gender

In [ ]:
# Individual loan splitup by gender
male = df['BORROWER_GENDERS']=='male'
female = df['']=='female'

plt.figure(figsize=(20,8))
sns.distplot(df[male]['LOAN_AMOUNT'], hist=False, color='blue', label = 'Men', kde_kws={'clip': (0, 4000)});
sns.distplot(df[female]['LOAN_AMOUNT'], hist=False, color='purple', label = 'Women', kde_kws={'clip': (0, 4000)});
plt.axvline(df[male]['LOAN_AMOUNT'].mean(), color="blue", linestyle="--")
plt.axvline(df[female]['LOAN_AMOUNT'].mean(), color="purple", linestyle="--")
plt.legend();

In [ ]:
# H0: mu_male - mu_female = 0
# Ha: mu_male - mu_female != 0

# alpha = 0.05

# T test
t, p = sts.ttest_ind(df[male]['LOAN_AMOUNT'], df[female]['LOAN_AMOUNT'], equal_var=False)

# p value
print("p-value: {:2.2f}".format(p))

### Question: Is there gender inequality in access to microfinance?
Possible approach to a response: Assuming the average loan amounts are a viable proxy for gender inequality in micro-finance, we can run t-tests to see if there are significant differences between men and women's average individual loan amount by country.

In [ ]:
# Step 1: State the null and alternative hypotheses
# H0: mu_male - mu_female = 0
# Ha: mu_male - mu_female != 0

# Step 2: Significance level: 
alpha = 0.05

# Steps 3 and 4: Compute t-tests and p-values
d = {}
for country in set(df['COUNTRY_NAME']):
    t, p = sts.ttest_ind(df[male & (df['COUNTRY_NAME'] == country)]['LOAN_AMOUNT'], df[female & (df['COUNTRY_NAME'] == country)]['LOAN_AMOUNT'], equal_var=False)
    d[country] = (p <= alpha)
d

# Step 5: Compare the p-value to alpha and decide.
# True: reject H0 that there is no difference between men and women.

## More EDA to look for other interesting questions...

In [ ]:
# Count Plot by Sector
plt.figure(figsize=(15,4))
sns.countplot(x='SECTOR_NAME', data=df, orient='h', order = df['SECTOR_NAME'].value_counts().index)
plt.xticks(rotation='vertical')
plt.ylabel('Number of loans')
plt.xlabel('Sector Name')
plt.title("Number of Loans By Sector");

In [ ]:
# Calculate correlations
# df[['SECTOR_NAME', 'COUNTRY_NAME']]

# Heatmap

def count_rows(x):
    return len(x)

g = df.groupby(['SECTOR_NAME','COUNTRY_NAME']).apply(count_rows).unstack()
plt.figure(figsize=(16,4))
sns.heatmap(g, cmap='Blues', linewidth=0.5);

In [ ]:
# By Activity
df['ACTIVITY_NAME'].value_counts(1).head(10)

## Model: Field Partners vs. Direct

In [ ]:
# Field Partners
plt.figure(figsize=(16,8))
sns.boxplot(x='COUNTRY_NAME', y='LOAN_AMOUNT', data=df[df['DISTRIBUTION_MODEL']=='field_partner'])
plt.xticks(rotation='vertical')
plt.ylim(0,10000);

In [ ]:
# Direct
plt.figure(figsize=(8,4))
sns.boxplot(x='COUNTRY_NAME', y='LOAN_AMOUNT', data=df[df['DISTRIBUTION_MODEL']=='direct'])
plt.xticks(rotation='vertical');
plt.ylim(0,10000);

In [ ]:
g_country = df.groupby('COUNTRY_NAME')
g_country['LOAN_AMOUNT'].sum().sort_values(ascending=False).head(10)

In [ ]:
df.sort_values?

In [ ]:
import seaborn as sns
corr = df[['LOAN_AMOUNT', 'LENDER_TERM', 'NUM_LENDERS_TOTAL', 'GROUP']].corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
           cmap='Blues')

In [ ]:
corr

In [ ]:
df[male].describe().T

In [ ]:
df[female].describe().transpose()

In [ ]:
g = sns.lmplot(x="LOAN_AMOUNT", y="NUM_LENDERS_TOTAL", hue = 'BORROWER_GENDERS',
               truncate=True, size=8, data=df[individual])
g.set_axis_labels("Loan Size", "Number of Lenders");

In [ ]:
g = sns.lmplot(x="LOAN_AMOUNT", y="NUM_LENDERS_TOTAL", hue='GROUP',
               truncate=True, size=5, data=df)
g.set_axis_labels("Loan Size", "Number of Lenders");

In [ ]:
df2 = pd.read_csv('data/loans_lenders.csv')

In [ ]:
df2.head()

In [ ]:
df.head(1)

In [ ]:
df_join = df.join(df2, on='LOAN_ID', how='left', lsuffix='LoanID')

In [ ]:
df_join.LENDERS.isnull().sum()

In [ ]:
d3 = pd.read_csv('data/lenders.csv')

In [ ]:
d3.head()

Other questions?

In [ ]:
d3.COUNTRY_CODE.value_counts(dropna=False)

# Story: What factors help the loans get fully funded faster?

## Step1. Distribtuion of the difference of post date and raised date

In [ ]:
df.head()

In [ ]:
df_clean = df.loc[df.notnull()["diff_posted_raised"],:]
df_clean = df_clean[df_clean.diff_posted_raised >= 0]

In [ ]:
df_error = df_clean[df_clean.diff_posted_raised < 0]

In [ ]:
df_clean_direct = df_clean[df_clean.DISTRIBUTION_MODEL=='direct']
df_clean_partner = df_clean[df_clean.DISTRIBUTION_MODEL=='field_partner']

In [ ]:
df_clean_partner.shape

In [ ]:
sns.distplot(df_clean_partner.diff_posted_raised)
plt.xlim(0, 100)

In [ ]:
df_clean_partner["diff_posted_planned"].hist(bins=60)

In [ ]:
df_clean.diff_posted_raised.describe()

**Median days to get posted loan fully raised are 7.8 days.
Mean is 12.6 days.**

In [ ]:
df_clean_partner["diff_posted_planned"].describe()

**Some (?) loans have a planned fund raising winder longer than 30 days. 
That explains why some loans take more than 30 days to get fully funded.**

In [ ]:
df_clean_partner[df_clean_partner.diff_posted_raised > df_clean_partner.diff_posted_planned+2]

In [ ]:
df_clean_partner.STATUS.value_counts()

## Step.2 Factors affect the speed for a loan to get fully funded

In [ ]:
plt.scatter(df_clean_partner['LOAN_AMOUNT'],df_clean_partner['diff_posted_raised'])
plt.xlim(0,55000)

In [ ]:
#sns.lmplot(x="LOAN_AMOUNT", y="diff_posted_raised",
#               truncate=True, size=5, data=df_clean_partner)

In [ ]:
(df_clean_partner[(df_clean_partner.LOAN_AMOUNT)>60000]['LOAN_AMOUNT']).hist()

In [ ]:
(df_clean_partner[(df_clean_partner.LOAN_AMOUNT)>20000]['LOAN_AMOUNT']).hist()

In [ ]:
(df_clean_partner[(df_clean_partner.LOAN_AMOUNT)<2000]['LOAN_AMOUNT']).hist()

In [ ]:
df_BKT1 = df_clean_partner[df_clean_partner.LOAN_AMOUNT <= 2000]

In [ ]:
df_BKT2 = df_clean_partner[(df_clean_partner.LOAN_AMOUNT > 2000) & (df_clean_partner.LOAN_AMOUNT <= 10000)]

In [ ]:
df_BKT3 = df_clean_partner[(df_clean_partner.LOAN_AMOUNT > 10000) & (df_clean_partner.LOAN_AMOUNT <= 50000)]

In [ ]:
df_BKT4 = df_clean_partner[(df_clean_partner.LOAN_AMOUNT > 50000) & (df_clean_partner.LOAN_AMOUNT <= 500000)]

In [ ]:
df_BKT1['LOAN_AMOUNT'].hist()

In [ ]:
df_BKT1['LOAN_AMOUNT'].describe()

In [ ]:
df_BKT2['LOAN_AMOUNT'].hist()

In [ ]:
df_BKT2['LOAN_AMOUNT'].describe()

In [ ]:
plt.scatter(df_BKT1['LOAN_AMOUNT'],df_BKT1['diff_posted_raised'])
# plt.xlim(0,55000)

In [ ]:
#plt.scatter(df_BKT1[df_BKT1['diff_posted_raised']<40]['LOAN_AMOUNT'],df_BKT1[df_BKT1['diff_posted_raised']<40]['diff_posted_raised'])


In [ ]:
# df_BKT2['LOAN_AMOUNT'].describe()

In [ ]:
df_BKT1_30=df_BKT1[df_BKT1['diff_posted_planned']<40]

In [ ]:
df_BKT1_30.shape

In [ ]:
plt.scatter(df_BKT1_30['diff_posted_raised'],df_BKT1_30['NUM_LENDERS_TOTAL'])

In [ ]:
df_BKT1_40=df_BKT1[df_BKT1['diff_posted_planned']>=40]

In [ ]:
df_BKT1_40.shape

In [ ]:
plt.scatter(df_BKT1_40['diff_posted_raised'],df_BKT1_40['NUM_LENDERS_TOTAL'])

In [ ]:
bins = list(range(0,2200,200))
df_BKT1['binned'] = pd.cut(df_BKT1['LOAN_AMOUNT'],bins)

In [ ]:
bins_post_plan = [30,35,40,60,80,106]
df_BKT1['bins_post_plan'] = pd.cut(df_BKT1['diff_posted_planned'],bins_post_plan)

In [ ]:
df_BKT1.head()

In [ ]:
df_BKT1.groupby('binned').agg({'diff_posted_raised':"mean"})

In [ ]:
df_b1_analysis = df_BKT1.groupby(['binned',"bins_post_plan"]).agg({'diff_posted_raised':"mean"}).unstack()
df_b1_analysis

In [ ]:
# corr = df[['LOAN_AMOUNT', 'LENDER_TERM', 'NUM_LENDERS_TOTAL', 'GROUP']].corr()
sns.heatmap(df_b1_analysis, 
            xticklabels=df_b1_analysis.columns.values,
            yticklabels=df_b1_analysis.index.values,
           cmap='Blues')

In [ ]:
df_BKT1.groupby(['SECTOR_NAME','binned','bins_post_plan']).agg({'diff_posted_raised':"mean"}).unstack()

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df_BKT1.groupby(['SECTOR_NAME','bins_post_plan']).agg({'diff_posted_raised':"mean"}).unstack())

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df_BKT1.groupby(['SECTOR_NAME','binned']).agg({'diff_posted_raised':"mean"}).unstack())


In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df_BKT1.groupby(['NUM_LENDERS_TOTAL','binned']).agg({'diff_posted_raised':"mean"}).unstack())

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df_BKT1.groupby(['COUNTRY_NAME','binned']).agg({'diff_posted_raised':"mean"}).unstack())

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df_BKT1[df_BKT1['GROUP']!=True].groupby(['BORROWER_GENDERS','binned']).agg({'diff_posted_raised':"mean"}).unstack())

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df_BKT1.groupby(['ORIGINAL_LANGUAGE','binned']).agg({'diff_posted_raised':"mean"}).unstack())

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df_BKT1[df_BKT1['GROUP']!=True].groupby(['SECTOR_NAME','BORROWER_GENDERS']).agg({'diff_posted_raised':"median"}).unstack())

In [ ]:
df_BKT1[df_BKT1['GROUP']!=True].groupby(['SECTOR_NAME','BORROWER_GENDERS']).agg({'diff_posted_raised':"median"}).unstack()

In [ ]:
# H0: mu_group - mu_individual = 0
# Ha: mu_group - mu_induvidual != 0

# alpha = 0.05

# T test
t, p = sts.ttest_ind(df_BKT1[df_BKT1['BORROWER_GENDERS']=='male']['diff_posted_raised'], df_BKT1[df_BKT1['BORROWER_GENDERS']=='female']['diff_posted_raised'], equal_var=False)

# p value
print("p-value: {:2.2f}".format(p))

In [ ]:
# H0: mu_group - mu_individual = 0
# Ha: mu_group - mu_induvidual != 0

# alpha = 0.05

# T test
t, p = sts.ttest_ind(df_BKT1[df_BKT1['SECTOR_NAME']=='Arts']['diff_posted_raised'], df_BKT1[df_BKT1['SECTOR_NAME']=='Clothing']['diff_posted_raised'], equal_var=False)

# p value
print("p-value: {:2.2f}".format(p))

In [ ]:
 # H0: mu_group - mu_individual = 0
# Ha: mu_group - mu_induvidual != 0

# alpha = 0.05

# T test
t, p = sts.ttest_ind(df_BKT1[df_BKT1['GROUP']==1]['diff_posted_raised'], df_BKT1[df_BKT1['GROUP']==0]['diff_posted_raised'], equal_var=False)

# p value
print("p-value: {:2.2f}".format(p))

In [ ]:
df_BKT1.corr()

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df_BKT1.corr())

## Other people's work


### similiar findings
- exclude direct, big companies 


### Scott's to calculte optimal bins for histgram




### t-test & p-value for gender / sector / group | individual 


- more men published video than women (proportion t-test)

- half of the countries 







we should filter by 'funded'

## see the trend day by day 